# Downloading data

## The MovieLens dataset

[Movielens](http://grouplens.org/datasets/movielens/) has a number of datasets:

This notebook is an example of downloading [a small dataset](http://files.grouplens.org/datasets/movielens/ml-latest-small.zip)

> Small: 100,000 ratings and 1,300 tag applications applied to 9,000 movies by 700 users. Last updated 10/2016.

### Step 1: instead of downloading the zipfile into the current directory, i want to save it in a /data directory:

In [1]:
import os

cwd = os.getcwd()
data_dir = cwd + "/data"
print(cwd, data_dir)

# if no data_dir make one:
if not os.path.isdir(data_dir):
    os.mkdir(data_dir, 755)

/Users/ko/git/cs109-2015 /Users/ko/git/cs109-2015/data


In [2]:
import urllib.request

url = "http://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
file_path = os.path.join(data_dir, "ml-latest-small.zip")

# only download if file doesn't exist:
if not os.path.isfile(file_path):
    print("Starting download...")
    urllib.request.urlretrieve(url, file_path)
    print("Downloaded", file_path)
else:
    print(f"File {file_path} already downloaded")

File /Users/ko/git/cs109-2015/data/ml-latest-small.zip already downloaded


Now to inspect and extract the downloaded zipile:

In [3]:
import zipfile

z = zipfile.ZipFile(file_path)

# this unzips the zipfile inside the data_dir
z.extractall(data_dir)
z.close()
os.listdir(data_dir)

['.DS_Store', 'ml-latest-small', 'ml-latest-small.zip']

The unzip process creates a directory with the same name as the zipfile inside the given path, so the actual files are:

In [62]:
csv_path = data_dir + "/ml-latest-small" 
os.listdir(csv_path)

['links.csv', 'movies.csv', 'ratings.csv', 'README.txt', 'tags.csv']

to list only the csv files where the data is:

In [5]:
[f for f in os.listdir(csv_path) if f[-4:]==".csv"]

['links.csv', 'movies.csv', 'ratings.csv', 'tags.csv']

### Step 2: take a look at the csv files

In [15]:
import pandas as pd

links = pd.read_csv(csv_path+'/links.csv')
movies = pd.read_csv(csv_path + "/movies.csv")
ratings = pd.read_csv(csv_path + "/ratings.csv")
tags = pd.read_csv(csv_path + "/tags.csv")

In [16]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [17]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [27]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [19]:
tags.head()

,userId,movieId,tag,timestamp
0,15,339,sandra 'boring' bullock,1138537770
1,15,1955,dentist,1193435061
2,15,7478,Cambodia,1170560997
3,15,32892,Russian,1170626366
4,15,34162,forgettable,1141391765


To select only the first two columbs of a df:

In [20]:
tags[tags.columns[:2]].head()

,userId,movieId
0,15,339
1,15,1955
2,15,7478
3,15,32892
4,15,34162


To filter data:

In [21]:
ratings[ratings['rating']> 3.0].head()

,userId,movieId,rating,timestamp
4,1,1172,4.0,1260759205
8,1,1339,3.5,1260759125
12,1,1953,4.0,1260759191
13,1,2105,4.0,1260759139
20,2,10,4.0,835355493


### Show comedy movies ranked >=4



To select only movies with comedy:

In [34]:
movies[movies["genres"].str.match("Comedy")].head(3)

,movieId,title,genres
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


The ratings table has multiple ratings for each movie, so to make that useful here we get the mean rating for each movie:

In [26]:
ratings_avg = ratings.groupby("movieId").mean()
ratings_avg.head()

,userId,rating,timestamp
movieId,,,
1,338.558704,3.872470,1.103116e+09
2,318.906542,3.401869,1.069321e+09
3,374.423729,3.161017,9.662429e+08
4,355.538462,2.384615,9.277797e+08
5,320.785714,3.267857,9.967201e+08


So now we have a dataframe of comedy movies and another of ratings, with both having the movieId column in common. So now to add ratings to all the movies. We can use [df.join](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) for this:

> `df1.join(df2,on=col1,how='inner')` - SQL-style joins the columns in df1 with the columns
on df2 where the rows for col have identical values. how can be one of 'left', 'right', 'outer', 'inner'

In [45]:
movies_ratings = movies.join(ratings_avg["rating"], on="movieId")
movies_ratings.head()

,movieId,title,genres,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.401869
2,3,Grumpier Old Men (1995),Comedy|Romance,3.161017
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615
4,5,Father of the Bride Part II (1995),Comedy,3.267857


## Now we can find movies with a particular rating easily

In [49]:
movies_ratings[movies_ratings["rating"] >= 4 ].head()

,movieId,title,genres,rating
25,26,Othello (1995),Drama,4.100000
27,28,Persuasion (1995),Drama|Romance,4.083333
28,29,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi,4.025000
29,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Crime|Drama,4.050000
39,41,Richard III (1995),Drama|War,4.021739


or high rated comedy movies:

In [59]:
movies_ratings[(movies_ratings["genres"].str.match("Comedy")) & (movies_ratings["rating"] >= 4.5)].head()

,movieId,title,genres,rating
157,178,Love & Human Remains (1993),Comedy|Drama,4.7
161,183,Mute Witness (1994),Comedy|Horror|Thriller,5.0
501,559,"Paris, France (1993)",Comedy,5.0
747,927,"Women, The (1939)",Comedy,4.5
795,984,"Pompatus of Love, The (1996)",Comedy|Drama,4.5
